### Imports

In [2]:
import pandas as pd
import numpy as np

import yfinance as yf
from ta.momentum import rsi

from pyhhmm.gaussian import GaussianHMM

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

In [14]:
# Get the data for the BTC-USD and DXY
ticker1 ='BTC-USD'
ticker2 = 'DX-Y.NYB'

df = yf.download(ticker1)
df_dxy= yf.download(ticker2)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### Feature Engineering

In [15]:
# Add DXY Close to BTC dataframe
df_dxy=df_dxy.loc[df_dxy.index >= '2014-09-17',"Close"]
df['DXY']=df_dxy
df.ffill(inplace=True)
df_close=df['Close']



In [16]:
# Check the head of the data
df.head(2)

,Open,High,Low,Close,Adj Close,Volume,DXY
Date,,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,84.699997
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,84.320000


In [17]:
# Add Returns and range to the dataframe
df['lrets'] = np.log(df['Close']/df['Close'].shift(1))
df['lrets_prod'] = df['lrets'].cumsum().apply(np.exp)-1
df['range'] = df['High']/df['Low'] -1

for i in range(1,8):
  df[f'lrets_T{i}'] = df['lrets'].shift(1)

df.dropna(inplace=True)

In [20]:
# Scaling the volume and dxy
scaler = MinMaxScaler()
df['Volume'] = scaler.fit_transform(df['Volume'].values.reshape(-1,1))
df['DXY'] = scaler.fit_transform(df['DXY'].values.reshape(-1,1))


In [21]:
# Check the head of the data
df.tail(3)


,Open,High,Low,Close,Adj Close,Volume,DXY,lrets,lrets_prod,range,lrets_T1,lrets_T2,lrets_T3,lrets_T4,lrets_T5,lrets_T6,lrets_T7
Date,,,,,,,,,,,,,,,,,
2024-02-05,42577.621094,43494.250000,42264.816406,42658.667969,42658.667969,0.053309,0.671986,0.001762,92.276832,0.029089,-0.009551,-0.009551,-0.009551,-0.009551,-0.009551,-0.009551,-0.009551
2024-02-06,42657.390625,43344.148438,42529.019531,43084.671875,43084.671875,0.047847,0.662818,0.009937,93.208326,0.019166,0.001762,0.001762,0.001762,0.001762,0.001762,0.001762,0.001762
2024-02-07,43090.019531,43654.441406,42781.453125,43532.953125,43532.953125,0.048779,0.658981,0.010351,94.188531,0.020406,0.009937,0.009937,0.009937,0.009937,0.009937,0.009937,0.009937


In [ ]:
# Save the data
df.to_csv('data/btc_with_features.csv')